# 1. Import libraries

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import datetime as dt
import json
from six.moves.urllib.request import urlopen

# 2. Collect the datasets of DublinBikes system

##2.1. Get the list of Bike Stations

In [ ]:
# UPDATE:
# From 26/8/2021, the API https://dublinbikes.staging.derilinx.com/api/v1
# has been shut down. You can not get data from the API.
# So please go to the new site to collect the data from the .CSV files:
# https://data.smartdublin.ie/dataset/dublinbikes-api


# Get the list of Bike Stations
url = 'https://dublinbikes.staging.derilinx.com/api/v1/resources/stations'

urlResponse = urlopen(url)

if hasattr(urlResponse.headers, 'get_content_charset'):
    encoding = urlResponse.headers.get_content_charset('utf-8')
else:
    encoding = urlResponse.headers.getparam('charset') or 'utf-8'

data = json.loads(urlResponse.read().decode(encoding))
stations_df = pd.DataFrame.from_dict(data, orient="columns")

In [ ]:
# Remove the Bike Station as a Terminal for testing
stations_df.drop(stations_df.tail(1).index, inplace=True)

# Remove irrelevant columns
stations_df.drop(columns={'st_CONTRACTNAME'}, inplace=True)

# Standardize the column names
stations_df.rename(columns={'st_ADDRESS':'address', 'st_ID':'station_id', \
                      'st_LATITUDE':'latitude', 'st_LONGITUDE':'longitude', \
                      'st_NAME':'name'}, inplace=True)

stations_df

,address,station_id,latitude,longitude,name
0,Clarendon Row,1,53.340927,-6.262501,CLARENDON ROW
1,Blessington Street,2,53.356770,-6.268140,BLESSINGTON STREET
2,Bolton Street,3,53.351181,-6.269859,BOLTON STREET
3,Greek Street,4,53.346874,-6.272976,GREEK STREET
4,Charlemont Street,5,53.330662,-6.260177,CHARLEMONT PLACE
...,...,...,...,...,...
111,Merrion Square South,113,53.338615,-6.248606,MERRION SQUARE SOUTH
112,Wilton Terrace (Park),114,53.333652,-6.248345,WILTON TERRACE (PARK)
113,Killarney Street,115,53.354843,-6.247579,KILLARNEY STREET
114,Broadstone,116,53.354698,-6.272314,BROADSTONE


##2.2. Define the function to get the historical data of a Bike Station from API

In [ ]:
# UPDATE:
# From 26/8/2021, the API https://dublinbikes.staging.derilinx.com/api/v1
# has been shut down. You can not get data from the API.
# So please go to the new site to collect the data from the .CSV files:
# https://data.smartdublin.ie/dataset/dublinbikes-api


def getDataOfAStation(_dfrom, _dto, _stationId):
  url = 'https://dublinbikes.staging.derilinx.com/api/v1/resources/historical/?dfrom=' \
  + _dfrom + '&dto=' + _dto + '&station=' + _stationId

  urlResponse = urlopen(url)
  if hasattr(urlResponse.headers, 'get_content_charset'):
      encoding = urlResponse.headers.get_content_charset('utf-8')
  else:
      encoding = urlResponse.headers.getparam('charset') or 'utf-8'

  data = json.loads(urlResponse.read().decode(encoding))
  if (len(data) > 0):
    if ('historic' in data[0]):
      df = pd.DataFrame.from_dict(data[0]['historic'], orient="columns")
      df.insert(loc=0, column='station_id', value = _stationId)
      if ('latitude' in data[0]):
        df.insert(loc=1, column='latitude', value = data[0]['latitude'])
      else:
        df.insert(loc=1, column='latitude', value = '')
      if ('longitude' in data[0]):  
        df.insert(loc=2, column='longitude', value = data[0]['longitude'])
      else:
        df.insert(loc=1, column='longitude', value = '')
      return df
  else:
    return None

## 2.3. Define a function to standardize the datetime value

In [ ]:
# 1. Split datetime string into date, hour, minute variables

# 2. To merge the historic Bike-availability data with the Weather, Covid data,
# we need to create a field "time_interval" with format: YYYYMMDDHHmm
# YYYY: Year; MM: Month; DD: Day; HH: Hour; mm: Minute
# Notes: mm with 5-minute interval (i.e., 00, 05, 10, ..., 55)

def transform_data(_df):
  _df['date'] = _df['time'].apply(lambda x: x[0:4] + x[5:7] + x[8:10])
  _df['hour'] = _df['time'].apply(lambda x: x[11:13])
  _df['minute'] = _df['time'].apply \
    (lambda x: \
      str(int(x[14:16]) - (int(x[14:16])%5)) \
      if (x[14] != '0') \
      else  '0'+str(int(x[14:16]) - (int(x[14:16])%5))
    )

  _df['time_interval'] = _df['date'] + _df['hour'] + _df['minute']
  _df.drop(columns=['time'], inplace=True)
  return _df

##2.4. Define a function to loop all Bike Stations for getting data

In [ ]:
def getDataAllStations(_stations, _dfrom, _dto):
  # Get data of the 1st Bike Station
  stationId = str(_stations.iloc[0]['station_id'])
  _df = getDataOfAStation(_dfrom, _dto, stationId)
  if _df is None:
    print('No data of the stationID =',stationId, 'between', _dfrom, 'and', _dto)
  else:
    _df = transform_data(_df)
    
  # Loop others in the list of Bike Station
  for i in range(1,len(_stations)):
    stationId = str(_stations.loc[i,'station_id'])
    print('index =',i,'. Getting data of the StationId =', stationId)
    while True:
      try:
        temp_df = getDataOfAStation(_dfrom, _dto, stationId)
        if temp_df is None:
          break
        else:
          temp_df = transform_data(temp_df)
        break
      except:
        print('API time out. Retrying...')
    if temp_df is None:
      print('No data of the stationID =',stationId, 'between', _dfrom, 'and', _dto)
    else:
      _df = pd.concat([_df, temp_df], ignore_index=True)
  return _df

##2.4. Get historical data for all Bike Stations

2.4.1. Datasets of the Quarter 1 & 2 in 2020

In [ ]:
# Get data from 01/01/2020 to 30/06/2020
dfrom = '201912312359'
dto = '202006302359'

df = getDataAllStations(stations_df, dfrom, dto)

# Save data into a file (It's stored in Google Drive for later analysis)
df.to_csv('drive/MyDrive/dublinbikes/datasets/dublinbikes_2020_Quarter_1_2.csv', index=False)

No data of the stationID = 1 between 201912312359 and 202006302359
index = 1 . Getting data of the StationId = 2
index = 2 . Getting data of the StationId = 3
index = 3 . Getting data of the StationId = 4
index = 4 . Getting data of the StationId = 5
index = 5 . Getting data of the StationId = 6
index = 6 . Getting data of the StationId = 7
index = 7 . Getting data of the StationId = 8
index = 8 . Getting data of the StationId = 9
index = 9 . Getting data of the StationId = 10
index = 10 . Getting data of the StationId = 11
index = 11 . Getting data of the StationId = 12
index = 12 . Getting data of the StationId = 13
index = 13 . Getting data of the StationId = 14
No data of the stationID = 14 between 201912312359 and 202006302359
index = 14 . Getting data of the StationId = 15
index = 15 . Getting data of the StationId = 16
index = 16 . Getting data of the StationId = 17
index = 17 . Getting data of the StationId = 18
index = 18 . Getting data of the StationId = 19
index = 19 . Getti

2.4.2. Datasets of the Quarter 3 & 4 in 2020

In [ ]:
# Get data from 01/07/2020 to 31/12/2020
dfrom = '202006302359'
dto = '202012312359'

df = getDataAllStations(stations_df, dfrom, dto)

# Save data into a file (It's stored in Google Drive for later analysis)
df.to_csv('drive/MyDrive/dublinbikes/datasets/dublinbikes_2020_Quarter_3_4.csv', index=False)

No data of the stationID = 1 between 202006302359 and 202012312359
index = 1 . Getting data of the StationId = 2
index = 2 . Getting data of the StationId = 3
index = 3 . Getting data of the StationId = 4
index = 4 . Getting data of the StationId = 5
index = 5 . Getting data of the StationId = 6
index = 6 . Getting data of the StationId = 7
index = 7 . Getting data of the StationId = 8
index = 8 . Getting data of the StationId = 9
index = 9 . Getting data of the StationId = 10
index = 10 . Getting data of the StationId = 11
index = 11 . Getting data of the StationId = 12
index = 12 . Getting data of the StationId = 13
index = 13 . Getting data of the StationId = 14
No data of the stationID = 14 between 202006302359 and 202012312359
index = 14 . Getting data of the StationId = 15
index = 15 . Getting data of the StationId = 16
index = 16 . Getting data of the StationId = 17
index = 17 . Getting data of the StationId = 18
index = 18 . Getting data of the StationId = 19
index = 19 . Getti

2.4.3. Datasets of the Quarter 1 & 2 in 2021

In [ ]:
# Get data from 01/01/2021 to 30/06/2021
dfrom = '202012312359'
dto = '202106302359'

df = getDataAllStations(stations_df, dfrom, dto)

# Save data into a file (It's stored in Google Drive for later analysis)
df.to_csv('drive/MyDrive/dublinbikes/datasets/dublinbikes_2021_Quarter_1_2.csv', index=False)

No data of the stationID = 1 between 202012312359 and 202106302359
index = 1 . Getting data of the StationId = 2
index = 2 . Getting data of the StationId = 3
index = 3 . Getting data of the StationId = 4
index = 4 . Getting data of the StationId = 5
index = 5 . Getting data of the StationId = 6
index = 6 . Getting data of the StationId = 7
index = 7 . Getting data of the StationId = 8
index = 8 . Getting data of the StationId = 9
index = 9 . Getting data of the StationId = 10
index = 10 . Getting data of the StationId = 11
index = 11 . Getting data of the StationId = 12
index = 12 . Getting data of the StationId = 13
index = 13 . Getting data of the StationId = 14
No data of the stationID = 14 between 202012312359 and 202106302359
index = 14 . Getting data of the StationId = 15
index = 15 . Getting data of the StationId = 16
index = 16 . Getting data of the StationId = 17
index = 17 . Getting data of the StationId = 18
index = 18 . Getting data of the StationId = 19
index = 19 . Getti